# 导言区

In [2]:
import numpy as np
from scipy.optimize import root
from CoolProp.CoolProp import PropsSI as psi
import scipy.constants as sc
from Functions.UnsteadyStateConduction import *
from Functions.Self_defined import check_Fo

## 例题04-01

In [3]:
N = 9
t = np.zeros((3, N))
# xpr1 = 8*t[0] + 2*t[1] + t[2] - 29
# xpr2 = t[0] + 5*t[1] + 2*t[2] - 32
# xpr3 = 2*t[0] + t[1] + 4*t[2] - 28
for i in range(N-1):
    t[0][i+1] = 1/8 * (29 - 2*t[1][i] - t[2][i])
    t[1][i+1] = 1/5 * (32 - t[0][i+1] - 2*t[2][i])
    t[2][i+1] = 1/4 * (28 - 2*t[0][i+1] - t[1][i+1])
print(t)

[[0.         3.625      1.73515625 1.86412598 1.98521817 2.00374495
  2.00151149 2.0001245  1.9999501 ]
 [0.         5.675      4.54546875 4.02875293 3.97865685 3.99416033
  3.99986273 4.00026367 4.00006125]
 [0.         3.76875    4.99605469 5.06074878 5.0127267  4.99958744
  4.99927858 4.99987183 5.00000964]]


## 例题04-02

In [9]:
t_a = 100
t_b = 500
t_c = 100
t_d = 100

N = 9
t = np.zeros((4, N))
t[0][0] = t[1][0] = 300
t[2][0] = t[3][0] = 200
for i in range(N-1):
    t[0][i+1] = 1/4 * (t_a + t_b + t[2][i] + t[1][i])
    t[1][i+1] = 1/4 * (t_b + t_c + t[0][i+1] + t[2][i])
    t[2][i+1] = 1/4 * (t_c + t_d + t[1][i+1] + t[3][i])
    t[3][i+1] = 1/4 * (t_d + t_a + t[2][i+1] + t[0][i+1])
print(t)

[[300.         275.         258.984375   252.70385742 250.75216293
  250.21956414 250.06397532 250.01850614 250.00536644]
 [300.         268.75       256.54296875 251.74407959 250.50418377
  250.14840923 250.04286684 250.01241597 250.00360406]
 [200.         167.1875     154.27246094 151.26457214 150.37407279
  150.10749204 150.03115772 150.00904981 150.00262326]
 [200.         160.546875   153.31420898 150.99210739 150.28155893
  150.08176405 150.02378326 150.00688899 150.00199743]]


## 例题04-03

In [10]:
d = 5e-2
l = 30e-2
t_0 = 30
t_oo = 1200
t = 800
h = 140
c = 0.48e3
rho = 7753
lambda_ = 33

r = d / 2
V = np.pi * r**2 * l
A = 2 * np.pi * r**2 + np.pi * d * l
l_c = V / A
Bi = get_Bi(l_c, lambda_, h)

if Bi < 0.05:
    print(f'可以使用集中参数法')
    Delta_T_ratio = (t - t_oo) / (t_0 - t_oo)
    a = get_a(lambda_, rho, c)
    tau_c = get_tau_c(l_c, lambda_, a, h)
    tau = - tau_c * np.log(Delta_T_ratio)
    print(f'tau = {tau:.0f} s')

可以使用集中参数法
tau = 329 s


## 例题03-04

In [5]:
delta = 100e-3
t_oo = 1000
t_0 = 20
t = 500

h = 174
lambda_ = 34.8
a = 0.555e-5
Bi_list = [0.1, 0.5, 1.0]
mu_list = [0.3111, 0.6533, 0.8603]
shape = 'P'

Bi = get_Bi(delta, lambda_, h)
mu = np.interp(Bi, Bi_list, mu_list)

x = delta
eta = x/delta

theta = t - t_oo
t_m = t_oo + theta / theta_to_theta_m_ratio(mu, eta, shape)
delta_T_m = abs(t - t_m)
print(f'剖面上的最大温差为：{delta_T_m:.2f} C')


def expressions(p):
    Fo = p
    xpr = (t - t_oo) / (t_0 - t_oo) - theta_to_theta_0_ratio(mu, eta, Fo, shape)
    return xpr

剖面上的最大温差为：129.66 C


## 例题03-05

In [6]:
d = 400e-3
t_0 = 20
t_oo = 900
t = 750

h = 174
lambda_ = 34.8
a = 0.695e-5
shape = 'C'

l_c = d / 2
eta = 1
Bi = h * l_c / lambda_
mu = get_mu(Bi, shape)

ratio_w_to_0 = (t - t_oo)/(t_0 - t_oo)


def expressions(p):
    Fo = p
    return ratio_w_to_0 - theta_to_theta_0_ratio(mu, eta, Fo, shape)


guess_values = np.ones(1)
Fo = root(expressions, guess_values).x[0]
tau = Fo * l_c**2 / a
print(f'所需的时间为：{tau:.0f} s')

check_Fo(Fo)

所需的时间为：5532 s


## 例题03-06

In [7]:
t_0 = 20
t_w = 1450
x = 80e-3
tau = 2 * 3600
a = 0.89e-6

t = t_x_for_constant_t_w(x, tau, t_0, t_w, a)
print(f't = {t:.2f} C')

t = 706.08 C


## 例题03-07

In [8]:
t_0 = 10    # 原地表温度，能否作为原地表以下的土壤的统一温度，有疑问
t_w = -15
tau = 45 * sc.day
t = 0
c = 1840
rho = 2050
lambda_ = 0.52

a = lambda_ / (rho * c)

guess_values = np.ones(1)
x = root(lambda x: t - t_x_for_constant_t_w(x, tau, t_0, t_w, a), guess_values).x[0]
print(f'x = {x:.2f} m')

x = 0.87 m


## 例题03-08

In [9]:
shape = 'P'
delta_1 = 0.5/2
delta_2 = 0.7/2
delta_3 = 1/2
lambda_ = 40.5
a = 0.722e-5
t_oo = 1200
tau = 4*3600
t_0 = 20
h = 348

Bi_1 = h * delta_1 / lambda_
Fo_1 = a * tau / delta_1**2
mu_1 = get_mu(Bi_1, shape)
ratio_w_to_0_1 = theta_to_theta_0_ratio(mu_1, 1, Fo_1, shape)
ratio_m_to_0_1 = theta_to_theta_0_ratio(mu_1, 0, Fo_1, shape)


Bi_2 = h * delta_2 / lambda_
Fo_2 = a * tau / delta_2**2
mu_2 = get_mu(Bi_2, shape)
ratio_w_to_0_2 = theta_to_theta_0_ratio(mu_2, 1, Fo_2, shape)
ratio_m_to_0_2 = theta_to_theta_0_ratio(mu_2, 0, Fo_2, shape)


Bi_3 = h * delta_3 / lambda_
Fo_3 = a * tau / delta_3**2
mu_3 = get_mu(Bi_3, shape)
ratio_w_to_0_3 = theta_to_theta_0_ratio(mu_3, 1, Fo_3, shape)
ratio_m_to_0_3 = theta_to_theta_0_ratio(mu_3, 0, Fo_3, shape)


ratio_m_to_0 = ratio_m_to_0_1 * ratio_m_to_0_2 * ratio_m_to_0_3
t_m = t_oo + ratio_m_to_0 * (t_0 - t_oo)
print(f'最低温度为：{t_m:.2f} C')

ratio_w_to_0 = ratio_w_to_0_1 * ratio_w_to_0_2 * ratio_w_to_0_3
t = t_oo + ratio_w_to_0 * (t_0 - t_oo)
print(f'最高温度为：{t:.2f} C')

check_Fo(Fo_1, Fo_2, Fo_3)

最低温度为：1157.66 C
最高温度为：1197.97 C


## 例题03-09

In [10]:
shape = ['P', 'C']
d = 600e-3
l = 1000e-3
t_0 = 30
t_oo = 1300
tau = 4*3600
h = 232
lambda_ = 40.5
a = 0.625e-5

# 先考虑厚度方向，作为无限大平板进行分析
l_c_1 = l / 2
Bi_1 = h * l_c_1 / lambda_
Fo_1 = a * tau / l_c_1**2
mu_1 = get_mu(Bi_1, shape[0])
eta_1 = 0
ratio_m_to_0_1 = theta_to_theta_0_ratio(mu_1, eta_1, Fo_1, shape[0])

# 再考虑径向，作为无限长圆柱进行分析
l_c_2 = d / 2
Bi_2 = h * l_c_2 / lambda_
Fo_2 = a * tau / l_c_2**2
mu_2 = get_mu(Bi_2, shape[1])
eta_2 = 0
ratio_m_to_0_2 = theta_to_theta_0_ratio(mu_2, eta_2, Fo_2, shape[1])

ratio_m_to_0 = ratio_m_to_0_1 * ratio_m_to_0_2
t_m = t_oo + ratio_m_to_0 * (t_0 - t_oo)
print(f't_m = {t_m:.2f} C')

check_Fo(Fo_1, Fo_2)

t_m = 1181.26 C


## 例题03-10

In [11]:
D = 4e-2
H = 6e-2
shape = ['P', 'C']
t_0 = 10
t_oo = 180
h = 15
t_m = 80
material = 'water'  # 用水的物性参数替代牛肉的物性参数
# 以（10°C+80°C)/2=45°C来确定从开始加热到中心温度为 80c水的物理特性，
# 以（10°C+180°C)/2=95°C来确定计算总加热量的物性。

# 先求出牛肉块的质量
T_0 = sc.convert_temperature(t_0, 'C', 'K')
rho_0 = psi('D', 'T', T_0, 'P', sc.atm, material)
V = np.pi * D**2 * H / 4
m = rho_0 * V

t_a = (t_0 + t_m) / 2
t_b = (t_0 + t_oo) / 2

T_a = sc.convert_temperature(t_a, 'C', 'K')
rho_a = psi('D', 'T', T_a, 'P', sc.atm, material)
c_a = psi('C', 'T', T_a, 'P', sc.atm, material)
lambda_a = psi('L', 'T', T_a, 'P', sc.atm, material)
a_a = lambda_a / (c_a * rho_a)


def expressions(p):
    tau = p
    # 先考虑厚度方向，作为无限大平板进行分析
    l_c_1 = H / 2
    Bi_1 = h * l_c_1 / lambda_a
    Fo_1 = a_a * tau / l_c_1**2
    mu_1 = get_mu(Bi_1, shape[0])
    eta_1 = 0
    ratio_m_to_0_1 = theta_to_theta_0_ratio(mu_1, eta_1, Fo_1, shape[0])

    # 再考虑径向，作为无限长圆柱进行分析
    l_c_2 = D / 2
    Bi_2 = h * l_c_2 / lambda_a
    Fo_2 = a_a * tau / l_c_2**2
    mu_2 = get_mu(Bi_2, shape[1])
    eta_2 = 0
    ratio_m_to_0_2 = theta_to_theta_0_ratio(mu_2, eta_2, Fo_2, shape[1])

    ratio_m_to_0 = ratio_m_to_0_1 * ratio_m_to_0_2
    xpr = t_m - t_oo - ratio_m_to_0 * (t_0 - t_oo)
    return xpr


guess_values = np.ones(1)
tau = root(expressions, guess_values).x[0]
print(f'tau = {tau:.0f} s')

# 先考虑厚度方向，作为无限大平板进行分析
l_c_1 = H / 2
Bi_1 = h * l_c_1 / lambda_a
Fo_1 = a_a * tau / l_c_1**2
mu_1 = get_mu(Bi_1, shape[0])
Q_to_Q_0_1 = Q_to_Q_0_ratio(mu_1, Fo_1, shape[0])

# 再考虑径向，作为无限长圆柱进行分析
l_c_2 = D / 2
Bi_2 = h * l_c_2 / lambda_a
Fo_2 = a_a * tau / l_c_2**2
mu_2 = get_mu(Bi_2, shape[1])
Q_to_Q_0_2 = Q_to_Q_0_ratio(mu_2, Fo_2, shape[1])

Q_to_Q_0 = Q_to_Q_0_1 + Q_to_Q_0_2 * (1 - Q_to_Q_0_1)

T_b = sc.convert_temperature(t_b, 'C', 'K')
rho_b = psi('D', 'T', T_b, 'P', sc.atm, material)
c_b = psi('C', 'T', T_b, 'P', sc.atm, material)
Q_0 = c_b * m * (t_oo - t_0)
Q = Q_0 * Q_to_Q_0
print(f'Q = {Q:.0f} J')

check_Fo(Fo_1, Fo_2)

tau = 1721 s
Q = 28067 J


## 例题03-11

In [12]:
t_0 = 25
t_w = 50
x = np.array([0.01, 0.1, 1.0, 10])
delta_t = 0.1
a = 1e-5
t = t_0 + delta_t

theta_ratio = (t - t_w) / (t_0 - t_w)
eta = root(lambda eta: theta_ratio - erf(eta), 1).x[0]
tau = (x/2/eta)**2/a

for i, tau_ in enumerate(tau):
    print(f'({i+1}) tau = {tau_:.0f} s')

(1) tau = 1 s
(2) tau = 60 s
(3) tau = 6036 s
(4) tau = 603587 s


## 例题03-12

In [13]:
T_oo = 303
f = 5
v = 20
d = 0.9e-3
rho = 8332
c_p = 188
Lambda_ = 51

# 涉及到对流换热公式，后续再求解。

## 例题03-13

In [14]:
q_0 = 2e4
t_0 = 20
a = 1e-7
lambda_ = 0.2
t_w_max = 180
tau = 30
x = 3e-3

t_0_tau = t_x_for_constant_q_0(0, tau, t_0, lambda_, a, q_0)
t_x_tau = t_x_for_constant_q_0(x, tau, t_0, lambda_, a, q_0)
print(f't_0_tau = {t_0_tau:.2f} C')
print(f't_x_tau = {t_x_tau:.2f} C')

t_0_tau = 215.44 C
t_x_tau = 46.12 C


## 例题03-14

In [15]:
d = 10e-2
height = 8e-2
t_0 = 40
t_oo = 105
tau = 80*60
shape = ['P', 'C']

# 物性参数可以选择水在温度为初始温度和最终温度的平均值时的物理参数
# 相变换热系数h很大，可认为h = np.inf
material = 'Water'
h = np.inf

# 先求出蔬菜罐头的质量
T_0 = sc.convert_temperature(t_0, 'C', 'K')
rho_0 = psi('D', 'T', T_0, 'P', sc.atm, material)
V = np.pi * d**2 / 4 * height
m = rho_0 * V


def expressions(p):
    t = p
    t_a = (t_0 + t)/2
    T_a = sc.convert_temperature(t_a, 'C', 'K')
    rho = psi('D', 'T', T_a, 'P', sc.atm, material)
    c = psi('C', 'T', T_a, 'P', sc.atm, material)
    lambda_ = psi('L', 'T', T_a, 'P', sc.atm, material)
    a = lambda_ / (rho * c)

    # 先考虑厚度方向，作为无限大平板进行分析
    l_c_1 = height / 2
    Bi_1 = h * l_c_1 / lambda_
    Fo_1 = a * tau / l_c_1**2
    mu_1 = get_mu(Bi_1, shape[0])
    eta_1 = 0
    ratio_m_to_0_1 = theta_to_theta_0_ratio(mu_1, eta_1, Fo_1, shape[0])
    Q_to_Q_0_1 = Q_to_Q_0_ratio(mu_1, Fo_1, shape[0])

    # 再考虑径向，作为无限长圆柱进行分析
    l_c_2 = d / 2
    Bi_2 = h * l_c_2 / lambda_
    Fo_2 = a * tau / l_c_2**2
    mu_2 = get_mu(Bi_2, shape[1])
    eta_2 = 0
    ratio_m_to_0_2 = theta_to_theta_0_ratio(mu_2, eta_2, Fo_2, shape[1])
    Q_to_Q_0_2 = Q_to_Q_0_ratio(mu_2, Fo_2, shape[1])

    ratio_m_to_0 = ratio_m_to_0_1 * ratio_m_to_0_2
    t_m = t_oo + ratio_m_to_0 * (t_0 - t_oo)
    xpr = t - t_m
    return xpr


guess_value = t_0 + 10
t = root(expressions, guess_value).x[0]
t_m = t
print(f't_m = {t_m:.2f} C')

t_a = (t_0 + t)/2
T_a = sc.convert_temperature(t_a, 'C', 'K')
rho = psi('D', 'T', T_a, 'P', sc.atm, material)
c = psi('C', 'T', T_a, 'P', sc.atm, material)
lambda_ = psi('L', 'T', T_a, 'P', sc.atm, material)
a = lambda_ / (rho * c)

# 先考虑厚度方向，作为无限大平板进行分析
l_c_1 = height / 2
Bi_1 = h * l_c_1 / lambda_
Fo_1 = a * tau / l_c_1 ** 2
mu_1 = get_mu(Bi_1, shape[0])
eta_1 = 0
ratio_m_to_0_1 = theta_to_theta_0_ratio(mu_1, eta_1, Fo_1, shape[0])
Q_to_Q_0_1 = Q_to_Q_0_ratio(mu_1, Fo_1, shape[0])

# 再考虑径向，作为无限长圆柱进行分析
l_c_2 = d / 2
Bi_2 = h * l_c_2 / lambda_
Fo_2 = a * tau / l_c_2 ** 2
mu_2 = get_mu(Bi_2, shape[1])
eta_2 = 0
ratio_m_to_0_2 = theta_to_theta_0_ratio(mu_2, eta_2, Fo_2, shape[1])
Q_to_Q_0_2 = Q_to_Q_0_ratio(mu_2, Fo_2, shape[1])

Q_to_Q_0 = Q_to_Q_0_1 + Q_to_Q_0_2 - Q_to_Q_0_1 * Q_to_Q_0_2
t_b = (t_0 + t_m)/2
T_b = sc.convert_temperature(t_b, 'C', 'K')
c_b = psi('C', 'T', T_b, 'P', sc.atm, material)
Q_0 = c_b * m * (t_oo - t_0)
Q = Q_0 * Q_to_Q_0
print(f'Q = {Q:.0f} J')

check_Fo(Fo_1, Fo_2)

t_m = 98.51 C
Q = 165211 J
